In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal import medfilt

from numpy.fft import fft, fftshift, fftfreq
from matplotlib import pyplot as plt
from pathlib import Path
import ffmpeg, librosa
import importlib

import helpers
importlib.reload(helpers)

# Driver

class Ex:
    def __init__(self, name: str, num_notes: int, spec_thresh: float, bpm: int, max_num_notes_per_beat: int):
        self.name = name
        self.num_notes = num_notes
        self.spec_thresh = spec_thresh
        self.bpm = bpm
        self.max_num_notes_per_beat = max_num_notes_per_beat
        self.min_time_between = 60 / (self.bpm * max_num_notes_per_beat)


exerciseNoteCounts = [
    Ex("ex1WholeMod.mp4", num_notes=90, spec_thresh=.15, bpm=80, max_num_notes_per_beat=2),
    Ex("ex1WholeModF.mp4", num_notes=90, spec_thresh=.15, bpm=120, max_num_notes_per_beat=2),
    Ex("ex2WholeMod.mp4", num_notes=49, spec_thresh=.15, bpm=80, max_num_notes_per_beat=1),
    Ex("ex3WholeMod.mp4", num_notes=145, spec_thresh=.15, bpm=100, max_num_notes_per_beat=2),
    Ex("ex3WholeModF.mp4", num_notes=145, spec_thresh=.15, bpm=130, max_num_notes_per_beat=2),
    Ex("ex4WholeMod.mp4", num_notes=102, spec_thresh=.15, bpm=90, max_num_notes_per_beat=4),
    Ex("ex4WholeModF.mp4", num_notes=102, spec_thresh=.15, bpm=120, max_num_notes_per_beat=2),
    Ex("ex5WholeMod.mp4", num_notes=133, spec_thresh=.15, bpm=64, max_num_notes_per_beat=2),
    Ex("ex5WholeModF.mp4", num_notes=133, spec_thresh=.15, bpm=86, max_num_notes_per_beat=2),
    Ex("ex6WholeMod.mp4", num_notes=118, spec_thresh=.15, bpm=70, max_num_notes_per_beat=4),
    Ex("ex6WholeModF.mp4", num_notes=118, spec_thresh=.15, bpm=90, max_num_notes_per_beat=4),
    Ex("ex7WholeMod.mp4", num_notes=86, spec_thresh=.15, bpm=70, max_num_notes_per_beat=2),
    Ex("ex8WholeMod.mp4", num_notes=112, spec_thresh=.15, bpm=55, max_num_notes_per_beat=4),
    Ex("ex8WholeModF.mp4", num_notes=112, spec_thresh=.15, bpm=80, max_num_notes_per_beat=4),
    Ex("ex9WholeMod.mp4", num_notes=121, spec_thresh=.15, bpm=100, max_num_notes_per_beat=4),
    Ex("ex10WholeMod.mp4", num_notes=102, spec_thresh=.15, bpm=120, max_num_notes_per_beat=3)
]

for ex in exerciseNoteCounts:
    exercise = ex.name
    note_count = ex.num_notes
    spec_thresh = ex.spec_thresh
    min_time_between = ex.min_time_between * .8
    print(f"min_time_between is {min_time_between}")

    ys, ts, sr = helpers.get_audio_data(exercise)

    ts_fr, fr_freq_amps = helpers.magnitudeSpectrogram(ys, ts, sr)
    ts_fr, spec_flux = helpers.computeSpectralFlux(ts_fr, fr_freq_amps, sr)

    plt.title(exercise)
    plt.xlabel("Time (s)"); plt.ylabel("Spectral Flux")
    plt.plot(ts_fr, spec_flux, c="orange")
    img_path = Path.cwd() / "spec_flux_graphs" / f"{exercise} Spectral Flux.png"
    plt.savefig(img_path)
    plt.xlim((5, 5.5))  
    plt.close()
    
    f0_frame_size = 2048
    f0_hop_len = 512
    f0, voiced, _ = librosa.pyin(y=ys, sr=sr, fmin=librosa.note_to_hz('E3'), fmax=librosa.note_to_hz('C6'), frame_length=f0_frame_size, hop_length=f0_hop_len)
    # f0 = gaussian_filter1d(f0, sigma=1)

    onsets = helpers.detect_onsets(spec_flux, ts_fr, threshold=spec_thresh, min_time_between=min_time_between, f0=f0, sr=sr, voiced=voiced, hop_len=f0_hop_len)

    print(f"For exercise {exercise}, Note count Actual:\t{note_count}, Note count Detected: \t{len(onsets)} | Accuracy: \t{min(note_count / len(onsets), len(onsets) / note_count)}")

min_time_between is 0.30000000000000004
For exercise ex1WholeMod.mp4, Note count Actual:	90, Note count Detected: 	88 | Accuracy: 	0.9777777777777777
min_time_between is 0.2
For exercise ex1WholeModF.mp4, Note count Actual:	90, Note count Detected: 	90 | Accuracy: 	1.0
min_time_between is 0.6000000000000001
For exercise ex2WholeMod.mp4, Note count Actual:	49, Note count Detected: 	46 | Accuracy: 	0.9387755102040817
min_time_between is 0.24
For exercise ex3WholeMod.mp4, Note count Actual:	145, Note count Detected: 	142 | Accuracy: 	0.9793103448275862
min_time_between is 0.18461538461538463
For exercise ex3WholeModF.mp4, Note count Actual:	145, Note count Detected: 	141 | Accuracy: 	0.9724137931034482
min_time_between is 0.13333333333333333
For exercise ex4WholeMod.mp4, Note count Actual:	102, Note count Detected: 	103 | Accuracy: 	0.9902912621359223
min_time_between is 0.2
For exercise ex4WholeModF.mp4, Note count Actual:	102, Note count Detected: 	64 | Accuracy: 	0.6274509803921569
min